In [17]:
import sys
sys.path.insert(0, '/home/claranunesbarrancos/XAI/CN/peepholelib/')

# torch stuff
import torch
import torchvision
from tensordict import MemoryMappedTensor as MMT

# our stuff
from peepholelib.datasets.cifar import Cifar
from peepholelib.models.model_wrap import ModelWrap 
from peepholelib.models.viz import viz_singular_values, viz_compare, viz_compare_per_layer_type
from peepholelib.coreVectors.coreVectors import CoreVectors 
from peepholelib.classifier.classifier_base import trim_corevectors
from peepholelib.classifier.tkmeans import KMeans as tKMeans 
from peepholelib.peepholes.peepholes import Peepholes

from nb_utils import *

# python stuff
import pandas as pd
from pathlib import Path as Path

In [18]:
use_cuda = torch.cuda.is_available()
cuda_index = 0
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(device)

cuda:0


In [ ]:
verbose = True
seed = 29
bs = 512 

model_dir = '/srv/newpenny/XAI/models'
model_name = 'CN_model=mobilenet_v2_dataset=CIFAR100_optim=Adam_scheduler=RoP_lr=0.001_factor=0.1_patience=5.pth'

ds_path = '/srv/newpenny/dataset/CIFAR100'
dataset = 'CIFAR100' 

cls_path = Path.cwd()/'../data/classifiers'


svds_path = Path.cwd()/'../data/data'
svds_name = 'svds' 

Dataset

In [20]:
ds = Cifar(
        data_path = ds_path,
        dataset=dataset
        )
ds.load_data(
        batch_size = bs,
        data_kwargs = {'num_workers': 8, 'pin_memory': True},
        seed = seed,
        )

dataset: CIFAR100


Model

In [1]:
nn = torchvision.models.mobilenet_v2(pretrained=True)

in_features = nn.classifier[-1].in_features
print("in features", in_features)
nn.classifier[-1] = torch.nn.Linear(in_features, len(ds.get_classes()))
model = ModelWrap(device=device)
model.set_model(model=nn, path=model_dir, name=model_name, verbose=verbose)

    
target_layers = [
    # initial conv
    # 'features.0.0',
    # 'features.0.1',
    # B1
    'features.1.conv.0.0',
    #'features.1.conv.1',
    # B2
    'features.2.conv.0.0',
    #'features.2.conv.1.0',
    'features.2.conv.2',
    'features.3.conv.0.0',
    #'features.3.conv.1.0',
    'features.3.conv.2',
    # B3
    'features.4.conv.0.0',
    #'features.4.conv.1.0',
    'features.4.conv.2',
    'features.5.conv.0.0',
    #'features.5.conv.1.0',
    'features.5.conv.2',
    'features.6.conv.0.0',
    #'features.6.conv.1.0',
    'features.6.conv.2',
    # B4
    'features.7.conv.0.0',
    #'features.7.conv.1.0',
    'features.7.conv.2',
    'features.8.conv.0.0',
    'features.8.conv.1.0',
    'features.8.conv.2',
    'features.9.conv.0.0',
    'features.9.conv.1.0',
    'features.9.conv.2',
    'features.10.conv.0.0',
    #'features.10.conv.1.0',
    'features.10.conv.2',
    # B5
    'features.11.conv.0.0',
    'features.11.conv.1.0',
    'features.11.conv.2',
    'features.12.conv.0.0',
    'features.12.conv.1.0',
    'features.12.conv.2',
    # 'features.13.conv.0.0',
    # 'features.13.conv.1.0',
    # 'features.13.conv.2',
    # B6
    'features.14.conv.0.0',
    'features.14.conv.1.0',
    'features.14.conv.2',
    'features.15.conv.0.0',
    'features.15.conv.1.0',
    'features.15.conv.2',
    # 'features.16.conv.0.0',
    # 'features.16.conv.1.0',
    # 'features.16.conv.2',
    # B7
    # 'features.17.conv.0.0',
    # 'features.17.conv.1.0',
    # 'features.17.conv.2',
    # final conv
    # 'features.18.0',
    # global avg pooling
    # 'features.18.1',
    # fully connected 
    # 'classifier.1',
]

model.set_target_layers(target_layers=target_layers, verbose=verbose)


NameError: name 'torchvision' is not defined

In [24]:
for key in nn.state_dict().keys():
    if '.weight' in key:
        print(key)

features.0.0.weight
features.0.1.weight
features.1.conv.0.0.weight
features.1.conv.0.1.weight
features.1.conv.1.weight
features.1.conv.2.weight
features.2.conv.0.0.weight
features.2.conv.0.1.weight
features.2.conv.1.0.weight
features.2.conv.1.1.weight
features.2.conv.2.weight
features.2.conv.3.weight
features.3.conv.0.0.weight
features.3.conv.0.1.weight
features.3.conv.1.0.weight
features.3.conv.1.1.weight
features.3.conv.2.weight
features.3.conv.3.weight
features.4.conv.0.0.weight
features.4.conv.0.1.weight
features.4.conv.1.0.weight
features.4.conv.1.1.weight
features.4.conv.2.weight
features.4.conv.3.weight
features.5.conv.0.0.weight
features.5.conv.0.1.weight
features.5.conv.1.0.weight
features.5.conv.1.1.weight
features.5.conv.2.weight
features.5.conv.3.weight
features.6.conv.0.0.weight
features.6.conv.0.1.weight
features.6.conv.1.0.weight
features.6.conv.1.1.weight
features.6.conv.2.weight
features.6.conv.3.weight
features.7.conv.0.0.weight
features.7.conv.0.1.weight
features.7.c

Visualize singular values 

In [25]:
direction = {'save_input':True, 'save_output':True}
model.add_hooks(**direction, verbose=False) 
    
dry_img, _ = ds._dss['train'][0]
dry_img = dry_img.reshape((1,)+dry_img.shape)
model.dry_run(x=dry_img)
dry_img, _ = ds._dss['train'][0]
dry_img = dry_img.reshape((1,)+dry_img.shape)

in out shapes:  torch.Size([32, 112, 112]) torch.Size([32, 112, 112])
in out shapes:  torch.Size([16, 112, 112]) torch.Size([96, 112, 112])
in out shapes:  torch.Size([96, 56, 56]) torch.Size([24, 56, 56])
in out shapes:  torch.Size([24, 56, 56]) torch.Size([144, 56, 56])
in out shapes:  torch.Size([144, 56, 56]) torch.Size([24, 56, 56])
in out shapes:  torch.Size([24, 56, 56]) torch.Size([144, 56, 56])
in out shapes:  torch.Size([144, 28, 28]) torch.Size([32, 28, 28])
in out shapes:  torch.Size([32, 28, 28]) torch.Size([192, 28, 28])
in out shapes:  torch.Size([192, 28, 28]) torch.Size([32, 28, 28])
in out shapes:  torch.Size([32, 28, 28]) torch.Size([192, 28, 28])
in out shapes:  torch.Size([192, 28, 28]) torch.Size([32, 28, 28])
in out shapes:  torch.Size([32, 28, 28]) torch.Size([192, 28, 28])
in out shapes:  torch.Size([192, 14, 14]) torch.Size([64, 14, 14])
in out shapes:  torch.Size([64, 14, 14]) torch.Size([384, 14, 14])
in out shapes:  torch.Size([384, 14, 14]) torch.Size([384

In [28]:
model.get_svds(path=svds_path, name=svds_name, verbose=verbose)
for k in model._svds.keys():
    for kk in model._svds[k].keys():
        print('svd shapes: ', k, kk, model._svds[k][kk].shape)

File /srv/newpenny/XAI/CN/XAI/src/../data/data/svds exists. Loading from disk.
Layers to compute SVDs:  ['features.1.conv.0.0', 'features.2.conv.0.0', 'features.2.conv.2', 'features.3.conv.0.0', 'features.3.conv.2', 'features.4.conv.0.0', 'features.4.conv.2', 'features.5.conv.0.0', 'features.5.conv.2', 'features.6.conv.0.0', 'features.6.conv.2', 'features.7.conv.0.0', 'features.7.conv.2', 'features.8.conv.0.0', 'features.8.conv.2', 'features.9.conv.0.0', 'features.9.conv.2', 'features.10.conv.0.0', 'features.10.conv.2', 'features.11.conv.0.0', 'features.11.conv.2', 'features.12.conv.0.0', 'features.12.conv.2', 'features.14.conv.0.0', 'features.14.conv.2', 'features.15.conv.0.0', 'features.15.conv.2']

 ---- Getting SVDs for features.1.conv.0.0

layer:  Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)


/home/claranunesbarrancos/XAI/CN/peepholelib/peepholelib/models/svd.py:135: UserWarning: Do not forget to pad your input accoding to the Conv2d padding. Deactivate this warning passing warns=False as argument.
  W_ = c2s(in_shape, layer, device=device)
/home/claranunesbarrancos/XAI/CN/peepholelib/peepholelib/models/svd.py:99: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  csr_mat = torch.sparse_csr_tensor(crow, cols, data, size=shape_out, device=device)



 ---- Getting SVDs for features.2.conv.0.0

layer:  Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.2.conv.2

layer:  Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.3.conv.0.0

layer:  Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.3.conv.2

layer:  Conv2d(144, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.4.conv.0.0

layer:  Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.4.conv.2

layer:  Conv2d(144, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.5.conv.0.0

layer:  Conv2d(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.5.conv.2

layer:  Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)

 ---- Getting SVDs for features.6.conv.0.0

layer:  Conv2d(32, 192, kernel_size=(

In [29]:
viz_singular_values(model, '/home/claranunesbarrancos/XAI/CN/XAI/viz_singular_values')

Visualize discrepancy in the populations of the clusters

In [ ]:
peep_layer = "features.16.conv.1.0"
cls_file = cls_path / f"{peep_layer}.empp.pt"  # Classifier file
if not cls_file.exists():
    raise FileNotFoundError(f"Classifier file {cls_file} not found!")

cls = torch.load(cls_file)  # Load trained classifier

# Count population per cluster
cluster_counts = torch.bincount(cls.assignments)  # Count samples per cluster

print(f"Cluster Population Distribution for {peep_layer}:")
for i, count in enumerate(cluster_counts):
    print(f"Cluster {i}: {count} samples")

# Optional: Plot the distribution
import matplotlib.pyplot as plt

plt.bar(range(len(cluster_counts)), cluster_counts.numpy())
plt.xlabel("Cluster ID")
plt.ylabel("Number of Samples")
plt.title(f"Cluster Population for {peep_layer}")
plt.show()
